In [7]:
from glob import glob
import scipy.io

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.patches as patches
import matplotlib.animation as animation
from IPython.core.display import display, HTML
from tqdm.notebook import tqdm
from IPython.display import clear_output
from PIL import ImageDraw, Image
import cv2
import re

import ntpath

In [2]:
def get_all_frame_files(path, vid_dir):
    tracked_files = glob('{}vidf1_*_people_full.mat'.format(path))
    
    ret = []
    
    for file in tracked_files:
        grabbed_name = re.search('(vidf1_33\_.*)\_people', ntpath.basename(file)).group(1)
        video_dir = '{}{}.y/'.format(vid_dir, grabbed_name)
        if len(glob('{}*'.format(video_dir))) == 0:
            #print("Dammit where is the vidf1 directory?")
            continue
        
        ret.append([
            file,
            video_dir
        ])
    return ret

In [3]:
def get_video_info(tracked_file, video_dir):
    frames_names = glob('{}*'.format(video_dir))
    frames_names.sort()
    
    print(video_dir, len(frames_names))
    
    frame_info = [{'file_name': name, 'persons': {}} for name in frames_names]
    
    mat = scipy.io.loadmat(tracked_file)
    for idx, person in enumerate(mat['people'][0]):
        
        for coordinates in person[0][0][0]:
            frame_info[int(coordinates[2])-1]['persons'][idx] = {'x': float(coordinates[0]), 'y': float(coordinates[1])}
    return frame_info

In [4]:
all_frame_files = get_all_frame_files('data/ucsdpeds/gt/vidf/', 'data/ucsdpeds/vidf/')

# The last videos were doing weird shit!!!
all_video_info = [get_video_info(i[0], i[1]) for i in all_frame_files]

video_info = all_video_info[4]

data/ucsdpeds/vidf/vidf1_33_007.y/ 200
data/ucsdpeds/vidf/vidf1_33_000.y/ 200
data/ucsdpeds/vidf/vidf1_33_001.y/ 200
data/ucsdpeds/vidf/vidf1_33_012.y/ 200
data/ucsdpeds/vidf/vidf1_33_019.y/ 200
data/ucsdpeds/vidf/vidf1_33_016.y/ 200
data/ucsdpeds/vidf/vidf1_33_006.y/ 200
data/ucsdpeds/vidf/vidf1_33_015.y/ 200
data/ucsdpeds/vidf/vidf1_33_014.y/ 200
data/ucsdpeds/vidf/vidf1_33_017.y/ 201
data/ucsdpeds/vidf/vidf1_33_013.y/ 200
data/ucsdpeds/vidf/vidf1_33_018.y/ 200
data/ucsdpeds/vidf/vidf1_33_010.y/ 200
data/ucsdpeds/vidf/vidf1_33_002.y/ 200
data/ucsdpeds/vidf/vidf1_33_003.y/ 200
data/ucsdpeds/vidf/vidf1_33_011.y/ 200
data/ucsdpeds/vidf/vidf1_33_008.y/ 200
data/ucsdpeds/vidf/vidf1_33_005.y/ 200
data/ucsdpeds/vidf/vidf1_33_009.y/ 200
data/ucsdpeds/vidf/vidf1_33_004.y/ 200


In [5]:
def generate_video(video_info):
    
    c_frames = len(video_info)

    fig,ax = plt.subplots(1, figsize=(14,8))
    fig.tight_layout()
    plt.axis('off')

    pbar = tqdm(total=c_frames)
    pbar.set_description("Generating frames")
    
    imgs = []
    for idx, frame in enumerate(video_info):
        img = Image.open(frame['file_name']).convert('RGB')
        
        draw = ImageDraw.Draw(img)

        for person_id in frame['persons']:
            region = frame['persons'][person_id]
            r = 2
            leftUpPoint = (region['x']-r, region['y']-r)
            rightDownPoint = (region['x']+r, region['y']+r)
            twoPointList = [leftUpPoint, rightDownPoint]
            draw.ellipse(twoPointList, fill=(255,0,0,255))

        imgs.append([ax.imshow(img)])
        pbar.update(1)
    pbar.close()

    pbar = tqdm(total=1)
    pbar.set_description("Stitching frames together")
    ani = animation.ArtistAnimation(fig, imgs, interval=100, blit=True)
    pbar.update(1)
    pbar.close()

    pbar = tqdm(total=1)
    pbar.set_description("Prepare for display")
    tag = ani.to_html5_video()
    pbar.update(1)
    pbar.close()
    plt.close()

    clear_output()
    display(HTML(tag))

In [6]:
generate_video(video_info)